In [1]:
import cv2
import numpy as np

In [2]:
#################################################
#                                               #
#            Вывод текста в видео               #
#                                               #
#################################################             

cap = cv2.VideoCapture('./data/peopleCounter.avi')

w = cap.get(3) # получаем ширину 
h = cap.get(4) # получаем высоту

mx = int(w/2)
my = int(h/2)

count = 0

while(cap.isOpened()):
    ok, frame = cap.read() # читаем кадр 
    
    try:
        count = count + 1
        text = 'Hello world' + str(count)
        cv2.putText(frame, text, (mx,my), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255), 1, cv2.LINE_AA)
        
        cv2.imshow('Frame', frame)
        
    except:
        # если не будет кадров, чтобы показывать 
        print('EOF')
        break
    k = cv2.waitKey(30) & 0xff  # выключение на Esc
    if k == 27:
        break
         
cap.release()  # высвободить cap
cv2.destroyAllWindows()


        

In [3]:
#################################################
#                                               #
#            Рисование линий в видео            #
#                                               #
################################################# 


cap = cv2.VideoCapture('./data/peopleCounter.avi')\

while(cap.isOpened()):
    ok, frame = cap.read()
    try:
        cv2.imshow('Frame', frame)
        frame2 = frame
    except:
        print('EOF')
        break
    line1 = np.array([[100,100],[300,100],[350,200]], np.int32).reshape((-1,1,2))
    line2 = np.array([[400,50],[450,300]], np.int32).reshape((-1,1,2))

    frame2 = cv2.polylines(frame2,[line1],False,(255,0,0),thickness=2)
    frame2 = cv2.polylines(frame2,[line2],False,(0,0,255),thickness=1)
    
    cv2.imshow('Frame2', frame2)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
        
cap.release()
cv2.destroyAllWindows()
    

In [4]:
#################################################
#                                               #
#            Вычитание заднего фона             #
#                                               #
################################################# 

cap = cv2.VideoCapture('./data/peopleCounter.avi')

fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=True) # создаем вычитатель заднего фона

while(cap.isOpened()):   
    ok, frame = cap.read()  
    
    fgmask = fgbg.apply(frame) # Используем вычитатель
    try:
        cv2.imshow('Frame', frame)
        cv2.imshow('BackgroundSubstructor', fgmask)
    except:
        print('EOF')
        break
    k = cv2.waitKey(30) & 0xff
    if k == 27: 
        break
cap.release()
cv2.destroyAllWindows()

In [26]:
#################################################
#                                               #
#         Морфологическая трансформация         #
#         Размывание, Растягивание              #
#         Открытие, Закрытие -                  #
#         (почередное применение                #
#                  размывания растягивание )    #
#                                               #
#################################################
import time

img = cv2.imread('./letters.jpg')
ret, thresh1 = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

kernel = np.ones((3,3), np.uint8)
kernel1 = np.ones((5,5), np.uint8)

erosion = cv2.erode(img, kernel, iterations=1) # приводит изображение к бинарному виду затемняет его ближе к черному 
dilation = cv2.dilate(img, kernel, iterations=1) # приводит изображение к бинарному виду осветляет его ближе к белому 

opening = cv2.morphologyEx(thresh1, cv2.MORPH_OPEN, kernel1)
closing = cv2.morphologyEx(thresh1, cv2.MORPH_CLOSE, kernel1)

cv2.imshow('Image', img)
cv2.imshow('Erosion', erosion)
cv2.imshow('Dilation', dilation)
cv2.imshow('Opening', opening)
cv2.imshow('Closing', closing)

cv2.waitKey(0)
cv2.destroyAllWindows() 
    



In [32]:
#################################################
#                                               #
#         Использование в счетчике людей        #
#         Морфологическая трансформация         #
#         Размывание, Растягивание              #
#         Открытие, Закрытие -                  #
#         (почередное применение                #
#                  размывания растягивание )    #
#                                               #
#################################################

cap = cv2.VideoCapture('./data/peopleCounter.avi')
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=True)

while(cap.isOpened()):
    ok, frame = cap.read()
    fgmask = fgbg.apply(frame)
    ret, thresh = cv2.threshold(fgmask, 127, 255, cv2.THRESH_BINARY)
    kernel = np.ones((3,3), np.uint8)
    try:
        frameOpening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)  # преобразование к черному 
        frameClosing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel) # преобразование к белому 
        cv2.imshow('Normal', frame)
        cv2.imshow('Opening', frameOpening)
        cv2.imshow('Closing', frameClosing)
    except:
        print('EOF')
        break
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

EOF


In [40]:
#################################################
#                                               #
#        Рисуем контур движ. объекта            #
#                                               #
#################################################


cap = cv2.VideoCapture('./data/peopleCounter.avi')
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=True)
kernelOp = np.ones((3,3), np.uint8)
kernelCl = np.ones((11,11), np.uint8)

while(cap.isOpened()):
    ok, frame = cap.read()
    fgmask = fgbg.apply(frame)
    try:
        ret, imBin = cv2.threshold(fgmask, 200, 255, cv2.THRESH_BINARY)
        mask = cv2.morphologyEx(imBin, cv2.MORPH_OPEN, kernelOp)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernelCl)
    except:
        print('EOF')
        break
    
    _, contours0, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL,\
                                               cv2.CHAIN_APPROX_NONE)
    for cnt in contours0:
        cv2.drawContours(frame, cnt, -1, (0,255,0), 3, 8)
    cv2.imshow('Mask', mask)
    cv2.imshow('Contours', frame)

    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
        
cap.release()
cv2.destroyAllWindows()
    


EOF


In [4]:
#################################################
#                                               #
#        Рисуем рамку движ. объекта             #
#                                               #
#                                               #
#################################################

# не могу брать видео, где камера не зафиксирована 

cap = cv2.VideoCapture('./data/peopleCounter.avi')
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=True)
kernelOp = np.ones((3,3), np.uint8)
kernelCl = np.ones((11,11), np.uint8)
areaTH = 800

while(cap.isOpened()):
    ok, frame = cap.read()
    #frame = cv2.resize(frame, (640,480))
    fgmask = fgbg.apply(frame)
    try:
        ret, imBin = cv2.threshold(fgmask, 200, 255, cv2.THRESH_BINARY)
        mask = cv2.morphologyEx(imBin, cv2.MORPH_OPEN, kernelOp)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernelCl)
    except:
        print('EOF')
        break
    
    _, contours0, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL,\
                                               cv2.CHAIN_APPROX_NONE)
    for cnt in contours0:
        cv2.drawContours(frame, cnt, -1, (0,255,0), 3, 8)
        area = cv2.contourArea(cnt)
        print(area)
        if area > areaTH:
            #################
            #   TRACKING    #
            #################            
            M = cv2.moments(cnt)
            print('moment', M)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            x,y,w,h = cv2.boundingRect(cnt)
            cv2.circle(frame,(cx,cy), 5, (0,0,255), -1)            
            img = cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)
    
    cv2.imshow('Contours', frame)

    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
        
cap.release()
cv2.destroyAllWindows()
    

400.0
3536.5
moment {'m00': 3536.5, 'm10': 172953.3333333333, 'm01': 925657.8333333333, 'm20': 10072547.25, 'm11': 45584678.95833333, 'm02': 243053436.0833333, 'm30': 653972237.5, 'm21': 2663711091.2666664, 'm12': 12048935848.833334, 'm03': 64020671501.55, 'mu20': 1614225.31839782, 'mu11': 315173.0058065057, 'mu02': 768005.7393884361, 'mu30': 3483923.8619599342, 'mu21': -3545435.651039779, 'mu12': -2636799.8568371534, 'mu03': 838874.2235870361, 'nu20': 0.12906747976873362, 'nu11': 0.02520006661211161, 'nu02': 0.061406895370203454, 'nu30': 0.004684189964147692, 'nu21': -0.004766893523841011, 'nu12': -0.0035452185283734276, 'nu03': 0.0011278794758442317}
22.0
73.0
357.5
722.5
28.5
17.0
27.0
10.0
11.0
6.0
14.0
18.5
44.0
6.0
18.5
8.5
10.0
4.0
15.0
29.5
28.5
1678.0
moment {'m00': 1678.0, 'm10': 85189.0, 'm01': 489154.8333333333, 'm20': 5310711.0, 'm11': 25358906.666666664, 'm02': 143343744.16666666, 'm30': 356952327.3, 'm21': 1600291052.6166666, 'm12': 7583408080.349999, 'm03': 42219320446.

1473.5
moment {'m00': 1473.5, 'm10': 31390.0, 'm01': 546525.5, 'm20': 996547.75, 'm11': 11563784.625, 'm02': 203481880.25, 'm30': 37125723.5, 'm21': 363220535.8, 'm12': 4275927143.5, 'm03': 76050208357.65001, 'mu20': 327845.9515609094, 'mu11': -78859.04313709028, 'mu02': 773958.8721581101, 'mu30': 1928029.0911137462, 'mu21': -3042095.088685766, 'mu12': -353321.4388946481, 'mu03': 4051001.869430542, 'nu20': 0.15099742622360926, 'nu11': -0.036320450173211775, 'nu02': 0.3564655812963119, 'nu30': 0.023133334200459783, 'nu21': -0.03650038408678419, 'nu12': -0.004239304771804929, 'nu03': 0.048605687810493396}
15.0
454.0
10.5
1776.0
moment {'m00': 1776.0, 'm10': 38473.0, 'm01': 668585.5, 'm20': 1155077.0, 'm11': 14431256.5, 'm02': 252505848.16666666, 'm30': 39960789.9, 'm21': 429022037.18333334, 'm12': 5431446027.383333, 'm03': 95667371013.55, 'mu20': 321646.97240990994, 'mu11': -52127.476069821045, 'mu02': 812959.1969312727, 'mu30': 1003174.2693395689, 'mu21': -3554952.01139161, 'mu12': 7286

1438.5
moment {'m00': 1438.5, 'm10': 30699.333333333332, 'm01': 574425.0, 'm20': 1020815.5833333333, 'm11': 11848677.625, 'm02': 230267465.25, 'm30': 41065303.5, 'm21': 383802723.21666664, 'm12': 4594729022.583333, 'm03': 92653428276.40001, 'mu20': 365654.60515390255, 'mu11': -410248.02672054246, 'mu02': 886804.4053701758, 'mu30': 3672836.985167064, 'mu21': -6321232.023498505, 'mu12': 8184548.455001518, 'mu03': -5726375.905014038, 'nu20': 0.17670600767676936, 'nu11': -0.19825619570290254, 'nu02': 0.4285565226155958, 'nu30': 0.046797967073932224, 'nu21': -0.08054286354037933, 'nu12': 0.10428457093495037, 'nu03': -0.07296342095716951}
6.0
2081.5
moment {'m00': 2081.5, 'm10': 898104.6666666666, 'm01': 74585.0, 'm20': 388168282.75, 'm11': 31727036.125, 'm02': 3523884.75, 'm30': 168060180974.30002, 'm21': 13513376743.816666, 'm12': 1485734638.6833334, 'm03': 201056296.20000002, 'mu20': 663121.9100075364, 'mu11': -454148.8681940064, 'mu02': 851330.2340259431, 'mu30': 5003480.351196289, 'mu21

31.5
2336.0
moment {'m00': 2336.0, 'm10': 971653.8333333333, 'm01': 200278.8333333333, 'm20': 404648935.8333333, 'm11': 83017169.25, 'm02': 18374666.166666664, 'm30': 168722520180.25, 'm21': 34447565901.066666, 'm12': 7594204698.666667, 'm03': 1804158525.95, 'mu20': 491756.11100411415, 'mu11': -288351.35952956975, 'mu02': 1203599.7790976986, 'mu30': 635301.2445373535, 'mu21': -5456945.939872414, 'mu12': 748590.2297477722, 'mu03': 22408602.20626664, 'nu20': 0.09011645283401298, 'nu11': -0.05284164468767037, 'nu02': 0.22056491072904788, 'nu30': 0.0024087826257616356, 'nu21': -0.02069033656506919, 'nu12': 0.002838324581820213, 'nu03': 0.08496355410316728}
425.0
72.0
2281.0
moment {'m00': 2281.0, 'm10': 946992.3333333333, 'm01': 201759.16666666666, 'm20': 393658924.5, 'm11': 83498301.08333333, 'm02': 18927210.333333332, 'm30': 163849472015.7, 'm21': 34594598734.75, 'm12': 7812473840.65, 'm03': 1884100032.25, 'mu20': 500450.4131472707, 'mu11': -265128.9969189912, 'mu02': 1081194.8427468464,

104.5
34.0
2652.5
moment {'m00': 2652.5, 'm10': 1045080.5, 'm01': 350451.6666666666, 'm20': 412643473.4166666, 'm11': 137469501.875, 'm02': 47509428.916666664, 'm30': 163280735881.55002, 'm21': 54038586976.666664, 'm12': 18553120757.6, 'm03': 6611970612.0, 'mu20': 883529.4090322852, 'mu11': -607860.2459550798, 'mu02': 1207309.908393547, 'mu30': 3689233.8067321777, 'mu21': -1400664.5953394324, 'mu12': -4891100.780295312, 'mu03': 15941927.59405899, 'nu20': 0.12557703502481132, 'nu11': -0.08639586483046223, 'nu02': 0.1715963239052727, 'nu30': 0.010181170213908878, 'nu21': -0.003865410924003767, 'nu12': -0.01349795978956347, 'nu03': 0.04399490161798472}
108.0
22.0
2686.0
moment {'m00': 2686.0, 'm10': 1053340.8333333333, 'm01': 364223.0, 'm20': 413930897.8333333, 'm11': 142256471.0, 'm02': 50620777.33333333, 'm30': 162999603832.55002, 'm21': 55675195066.25, 'm12': 19691106208.516666, 'm03': 7212281355.5, 'mu20': 853120.0346136689, 'mu11': -577095.0235169828, 'mu02': 1231948.692603618, 'mu30

2410.5
moment {'m00': 2410.5, 'm10': 891046.0, 'm01': 430213.1666666666, 'm20': 329788015.9166666, 'm11': 158772607.2083333, 'm02': 78070459.08333333, 'm30': 122210985237.90001, 'm21': 58665950459.7, 'm12': 28764247212.13333, 'm03': 14418447160.35, 'mu20': 411133.8938497901, 'mu11': -256524.21903306246, 'mu02': 1288310.6604496539, 'mu30': 252670.5319519043, 'mu21': -3204030.1075760573, 'mu12': -3083255.061862588, 'mu03': 24986947.478729248, 'nu20': 0.07075693461685056, 'nu11': -0.04414831193750367, 'nu02': 0.22172074482609402, 'nu30': 0.0008857002252659664, 'nu21': -0.011231266923438576, 'nu12': -0.01080790736358579, 'nu03': 0.08758815220617804}
2322.5
moment {'m00': 2322.5, 'm10': 854211.6666666666, 'm01': 423106.3333333333, 'm20': 314579928.9166666, 'm11': 155363579.625, 'm02': 78291016.91666666, 'm30': 115998419703.6, 'm21': 57119200019.083336, 'm12': 28699206492.283333, 'm03': 14727485650.0, 'mu20': 402287.81033962965, 'mu11': -254231.4294193387, 'mu02': 1210728.7329087257, 'mu30':

4.0
83.0
2779.5
moment {'m00': 2779.5, 'm10': 956793.1666666666, 'm01': 630604.8333333333, 'm20': 330175697.5833333, 'm11': 216764607.2083333, 'm02': 143803151.5833333, 'm30': 114220692565.15001, 'm21': 74695374809.68333, 'm12': 49358775793.71667, 'm03': 32962955048.050003, 'mu20': 816761.1631038785, 'mu11': -309828.98533657193, 'mu02': 733370.751039356, 'mu30': 1302658.171722412, 'mu21': -615920.0322033167, 'mu12': -2301187.3029886484, 'mu03': 4546443.093212128, 'nu20': 0.10572111192028609, 'nu11': -0.04010409200951496, 'nu02': 0.09492710323670854, 'nu30': 0.0031982600426585005, 'nu21': -0.0015121944276940916, 'nu12': -0.005649828605527682, 'nu03': 0.011162335290166662}
15.5
24.0
64.5
2679.5
moment {'m00': 2679.5, 'm10': 915583.1666666666, 'm01': 619023.3333333333, 'm20': 313676916.9166666, 'm11': 211162066.54166666, 'm02': 143797068.41666666, 'm30': 107747901239.95001, 'm21': 72221742792.2, 'm12': 48966238001.4, 'm03': 33592725242.100002, 'mu20': 822826.5701985955, 'mu11': -357748.26

In [3]:
#################################################
#                                               #
#             Индентифицируем людей             #
#                                               #
#                                               #
#################################################
import Person

cap = cv2.VideoCapture('./data/peopleCounter.avi')
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows=True)

kernelOp = np.ones((3,3), np.uint8)
kernelCl = np.ones((9,9), np.uint8)
font = cv2.FONT_HERSHEY_SIMPLEX
persons = []
max_p_age = 5
pid = 1
minArea = 1228

while(cap.isOpened()):
    ok, frame = cap.read()
    fgmask = fgbg.apply(frame)
    
    try:
        # убираем тени и другте помехи после вычитателя заднего плана 
        # оставляем только черное и белое 
        ret, imBin = cv2.threshold(fgmask, 180, 255, cv2.THRESH_BINARY)
        
        # Cначала сужается, а затем расширяется. 
        # Обычно используется для подсчёта регионов на двоичном изображении.
        mask = cv2.morphologyEx(imBin, cv2.MORPH_OPEN, kernelOp)
        
        # Cначала расширяется, а затем сужается. 
        # Обычно используется для уменьшения шумовых выбросов на границах регионов.
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernelCl)        
    except:
        print('EOF')
        break
        
    # нахождение контуров движ элементов    
    _, contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL,
                                              cv2.CHAIN_APPROX_NONE)
    for contr in contours:
        #cv2.drawContours(frame, contr, -1, (0,255,0), 1, 8)
        
        # значение контурной областа 
        area = cv2.contourArea(contr)
        
        
        if area > minArea:
            
            # получение моментов движ элементов
            M = cv2.moments(contr)
            
            # нахождение центра области 
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            
            # нахождение прямоугольной области 
            x, y, w, h = cv2.boundingRect(contr)
            
            ######################
            new = True
            for i in persons: 
                if abs(x-i.getX()) <= w and abs(y-i.getY()) <= h:
                    new = False
                    i.updateCoords(cx,cy)
                    break
            if new == True:
                p = Person.MyPerson(pid, cx, cy, max_p_age)
                persons.append(p)
                pid += 1
            ######################
            
            # рисуем точку в центре области 
            cv2.circle(frame, (cx,cy), 5, (0,0,255), -1)
            img = cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)            
            cv2.drawContours(frame, contr, -1, (0,255,0), 1)
    
    for i in persons:
        if len(i.getTracks()) >= 2:
            pts = np.array(i.getTracks(), np.int32)
            pts = pts.reshape((-1,1,2))
            frame = cv2.polylines(frame,[pts],False,i.getRGB())
        if i.getId() == 9:
            print(str(i.getX()), ',', str(i.getY()))
        cv2.putText(frame, str(i.getId()),(i.getX(),i.getY()),font,1,i.getRGB(),1,cv2.LINE_AA)        
        
    cv2.imshow('frame', frame)
    print(pid)
        
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
cap.release()
cv2.destroyAllWindows()
        


1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


In [3]:
import numpy as np
import cv2
import Person
import time


cnt_up   = 0
cnt_down = 0

cap = cv2.VideoCapture('./data/peopleCounter.avi')

w = cap.get(3)
h = cap.get(4)
frameArea = h*w
areaTH = frameArea/250
print('Area Threshold', areaTH)

#Lineas de entrada/salida
line_up = int(2*(h/5))
line_down   = int(3*(h/5))

up_limit =   int(1*(h/5))
down_limit = int(4*(h/5))

print("Red line y:",str(line_down))
print("Blue line y:", str(line_up))
line_down_color = (255,0,0)
line_up_color = (0,0,255)
pt1 =  [0, line_down];
pt2 =  [w, line_down];
pts_L1 = np.array([pt1,pt2], np.int32)
pts_L1 = pts_L1.reshape((-1,1,2))
print(pts_L1)
pt3 =  [0, line_up];
pt4 =  [w, line_up];
pts_L2 = np.array([pt3,pt4], np.int32)
pts_L2 = pts_L2.reshape((-1,1,2))

pt5 =  [0, up_limit];
pt6 =  [w, up_limit];
pts_L3 = np.array([pt5,pt6], np.int32)
pts_L3 = pts_L3.reshape((-1,1,2))
pt7 =  [0, down_limit];
pt8 =  [w, down_limit];
pts_L4 = np.array([pt7,pt8], np.int32)
pts_L4 = pts_L4.reshape((-1,1,2))

#Substractor de fondo
fgbg = cv2.createBackgroundSubtractorMOG2(detectShadows = True)

#Elementos estructurantes para filtros morfoogicos
kernelOp = np.ones((3,3),np.uint8)
kernelOp2 = np.ones((5,5),np.uint8)
kernelCl = np.ones((11,11),np.uint8)

#Variables
font = cv2.FONT_HERSHEY_SIMPLEX
persons = []
max_p_age = 5
pid = 1

while(cap.isOpened()):
    ret, frame = cap.read()
    for i in persons:
        i.age_one() #age every person one frame
    #########################
    #   PRE-PROCESAMIENTO   #
    #########################
    
    #Aplica substraccion de fondo
    fgmask = fgbg.apply(frame)
    fgmask2 = fgbg.apply(frame)

    #Binariazcion para eliminar sombras (color gris)
    try:
        ret,imBin= cv2.threshold(fgmask,200,255,cv2.THRESH_BINARY)
        ret,imBin2 = cv2.threshold(fgmask2,200,255,cv2.THRESH_BINARY)
        #Opening (erode->dilate) para quitar ruido.
        mask = cv2.morphologyEx(imBin, cv2.MORPH_OPEN, kernelOp)
        mask2 = cv2.morphologyEx(imBin2, cv2.MORPH_OPEN, kernelOp)
        #Closing (dilate -> erode) para juntar regiones blancas.
        mask =  cv2.morphologyEx(mask , cv2.MORPH_CLOSE, kernelCl)
        mask2 = cv2.morphologyEx(mask2, cv2.MORPH_CLOSE, kernelCl)
    except:
        print('EOF')
        print('UP:',cnt_up)
        print('DOWN:',cnt_down)
        break
    #################
    #   CONTORNOS   #
    #################
    
    # RETR_EXTERNAL returns only extreme outer flags. All child contours are left behind.
    _, contours0, hierarchy = cv2.findContours(mask2,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    for cnt in contours0:
        area = cv2.contourArea(cnt)
        if area > areaTH:
            #################
            #   TRACKING    #
            #################
            
            #Falta agregar condiciones para multipersonas, salidas y entradas de pantalla.
            
            M = cv2.moments(cnt)
            cx = int(M['m10']/M['m00'])
            cy = int(M['m01']/M['m00'])
            x,y,w,h = cv2.boundingRect(cnt)

            new = True
            if cy in range(up_limit,down_limit):
                for i in persons:
                    if abs(cx-i.getX()) <= w and abs(cy-i.getY()) <= h:
                        # el objeto esta cerca de uno que ya se detecto antes
                        new = False
                        i.updateCoords(cx,cy)   #actualiza coordenadas en el objeto and resets age
                        if i.going_UP(line_down,line_up) == True:
                            cnt_up += 1;
                            print("ID:",i.getId(),'crossed going up at',time.strftime("%c"))
                        elif i.going_DOWN(line_down,line_up) == True:
                            cnt_down += 1;
                            print("ID:",i.getId(),'crossed going down at',time.strftime("%c"))
                        break
                    if i.getState() == '1':
                        if i.getDir() == 'down' and i.getY() > down_limit:
                            i.setDone()
                        elif i.getDir() == 'up' and i.getY() < up_limit:
                            i.setDone()
                    if i.timedOut():
                        #sacar i de la lista persons
                        index = persons.index(i)
                        persons.pop(index)
                        del i     #liberar la memoria de i
                if new == True:
                    p = Person.MyPerson(pid,cx,cy, max_p_age)
                    persons.append(p)
                    pid += 1     
            #################
            #   DIBUJOS     #
            #################
            cv2.circle(frame,(cx,cy), 5, (0,0,255), -1)
            img = cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)            
            #cv2.drawContours(frame, cnt, -1, (0,255,0), 3)
            
    #END for cnt in contours0
            
    #########################
    # DIBUJAR TRAYECTORIAS  #
    #########################
    for i in persons:
        if len(i.getTracks()) >= 2:
            pts = np.array(i.getTracks(), np.int32)
            pts = pts.reshape((-1,1,2))
            frame = cv2.polylines(frame,[pts],False,i.getRGB())
       
        cv2.putText(frame, str(i.getId()),(i.getX(),i.getY()),font,1,i.getRGB(),1,cv2.LINE_AA)
        
    #################
    #   IMAGANES    #
    #################
    str_up = 'UP: '+ str(cnt_up)
    str_down = 'DOWN: '+ str(cnt_down)
    frame = cv2.polylines(frame,[pts_L1],False,line_down_color,thickness=2)
    frame = cv2.polylines(frame,[pts_L2],False,line_up_color,thickness=2)
    frame = cv2.polylines(frame,[pts_L3],False,(255,255,255),thickness=1)
    frame = cv2.polylines(frame,[pts_L4],False,(255,255,255),thickness=1)
    cv2.putText(frame, str_up ,(10,40),font,0.5,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(frame, str_up ,(10,40),font,0.5,(0,0,255),1,cv2.LINE_AA)
    cv2.putText(frame, str_down ,(10,90),font,0.5,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(frame, str_down ,(10,90),font,0.5,(255,0,0),1,cv2.LINE_AA)

    cv2.imshow('Frame',frame)
    #cv2.imshow('Mask',mask)    
    
    #preisonar ESC para salir
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
#END while(cap.isOpened())
    
#################
#   LIMPIEZA    #
#################
cap.release()
cv2.destroyAllWindows()

Area Threshold 1228.8
Red line y: 288
Blue line y: 192
[[[  0 288]]

 [[640 288]]]
ID: 1 crossed going down at Tue May  1 11:44:50 2018
ID: 2 crossed going down at Tue May  1 11:44:55 2018
ID: 3 crossed going down at Tue May  1 11:44:59 2018
ID: 5 crossed going up at Tue May  1 11:45:01 2018
ID: 7 crossed going down at Tue May  1 11:45:02 2018
ID: 11 crossed going down at Tue May  1 11:45:03 2018
ID: 9 crossed going up at Tue May  1 11:45:03 2018
ID: 7 crossed going up at Tue May  1 11:45:04 2018
ID: 13 crossed going down at Tue May  1 11:45:08 2018
EOF
UP: 3
DOWN: 6
